# Task 1

Calculated demographic parity, equal opportunity and predictive rate parity coefficients:

In [16]:
groups_df

,Blue Group,Red Group,Coefficient
Demographic Parity,0.650000,0.5,1.300000
Equal Opportunity,0.750000,0.5,1.500000
Positive Predictive Parity,0.923077,0.5,1.846154


All coefficients are above 1.25. Therefore, Red Group is strongly unprivileged with the decision rule

Starred Task: From the lecture, we know that:
```
The Fairness Trade-off (the impossibility theorem)
- Except for trivial cases all these criteria cannot be satisfied jointly
- In fact, each two out of {Sufficiency, Separation, Independence} are mutually exclusive.
```
Nevertheless, changing the enrollment probability in the red group to 0.65 will change the demographic parity coefficient to 1. Thus, the rule is improved.

# Task 2

## 1
I selected the "Adult Income" dataset for my analysis, where the target variable is 'income'—categorized as 'above' or 'below' $50,000 in earnings per year. The dataset consists of 37,155 samples with earnings below $50,000 and 11,687 samples with earnings above $50,000. For this analysis, 'gender' is considered as a protected attribute, and the dataset contains 32,650 samples of men and 16,192 samples of women.

## 2 and 3
Next, I trained LogisticRegression from Sklearn and XGBoost. I got following fairness results:

In [19]:
logistic_fairness_df

,Female,Male,Coefficient
Demographic Parity,0.696233,0.890749,0.781627
Equal Opportunity,0.748124,0.917114,0.815737
Positive Predictive Parity,0.965071,0.970464,0.994443


In [22]:
xgb_fairness_df

,Female,Male,Coefficient
Demographic Parity,0.696233,0.890749,0.781627
Equal Opportunity,0.883213,0.917114,0.963035
Positive Predictive Parity,0.934850,0.970464,0.963302


The XGBoost got better equal opportunity coefficient and slighly worse positive predictive parity than Logistic Regression.

## 4
Next, as bias mitigation technique I selected dataset balancing. I upsample women samples to make their number equal with men. In the result balanced dataset is bigger than normal (65300 samples vs 48842).
The logistic regression was trained on this balanced dataset and fairness results were following:

In [24]:
logistic_balanced_fairness_df

,Female,Male,Coefficient
Demographic Parity,0.696233,0.890749,0.781627
Equal Opportunity,0.743378,0.917114,0.810562
Positive Predictive Parity,0.974089,0.970464,1.003736


Unfortunately, the results for the logistic regression model trained on the balanced dataset are similar to those of the model trained on the imbalanced dataset. This suggests that balancing the dataset may not always lead to better fairness scores, and other techniques should be considered in such cases.

## 5
I've chosen f1 score and accuracy as metrics for the problem.

In [25]:
results_df

Metric,accuracy_score,f1_score
Model,,
Logistic Regression,0.797899,0.380857
Logistic Regression on balanced dataset,0.797510,0.356120
XGBoost,0.896278,0.765723


XGBoost had the best score, but it's considered as better and more sophisticated model than logistic regression. The suprise is that Logistic regression on balanced dataset gained worse results than casual logistic regression. There is no correlation between performance and fairness scores because logistic regression and XGBoost had similar results in fairness but different on performance.

# Appendix

## Task 1

In [2]:
import numpy as np
import pandas as pd

import os

while 'Homeworks' in os.getcwd():
    os.chdir('..')

In [3]:
blue_group = np.array([
    [60, 5],
    [20, 15]
])

red_group = np.array([
    [1, 1],
    [1, 1]
])

In [4]:
def demographic_parity(group: np.array) -> float:
    return group[0].sum() / group.sum()

def equal_opportunity(group: np.array) -> float:
    return group[0][0] / group[:, [0]].sum()

def positive_predictive_parity(group: np.array) -> float:
    return group[0][0] / group[0].sum()


In [5]:
functions = [
    demographic_parity,
    equal_opportunity,
    positive_predictive_parity,
]

blue_results = [f(blue_group) for f in functions]
red_results = [f(red_group) for f in functions]

# make dataframe with results
groups_df = pd.DataFrame([blue_results, red_results], columns=["Demographic Parity", "Equal Opportunity", "Positive Predictive Parity"], index=["Blue Group", "Red Group"])
groups_df = groups_df.T
groups_df["Coefficient"] = groups_df["Blue Group"] / groups_df["Red Group"]
groups_df

,Blue Group,Red Group,Coefficient
Demographic Parity,0.650000,0.5,1.300000
Equal Opportunity,0.750000,0.5,1.500000
Positive Predictive Parity,0.923077,0.5,1.846154


All coefficients are above 1.25. Therefore, Red Group is strongly unprivileged with the decision rule

Starred Task: From the lecture, we know that:
```
The Fairness Trade-off (the impossibility theorem)
- Except for trivial cases all these criteria cannot be satisfied jointly
- In fact, each two out of {Sufficiency, Separation, Independence} are mutually exclusive.
```
Nevertheless, changing the enrollment probability in the red group to 0.65 will change the demographic parity coefficient to 1. Thus, the rule is improved.

## Task 2

In [6]:
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, f1_score, accuracy_score


In [7]:
income_df = pd.read_csv('./Homeworks/HW2/JanPiotrowski/adult.csv')
income_df

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
48837,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
48838,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
48839,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
48840,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K


In [8]:
income_df.gender.value_counts()

gender
Male      32650
Female    16192
Name: count, dtype: int64

In [9]:
income_df.income.value_counts()

income
<=50K    37155
>50K     11687
Name: count, dtype: int64

In [10]:
def prepare_X(df):
    df = df.copy()
    df = df.drop(columns=['income', "native-country"])
    df = pd.get_dummies(df, drop_first=True)
    return df

def prepare_y(df):
    return df['income'].map({'<=50K': 0, '>50K': 1})

X = prepare_X(income_df)
y = prepare_y(income_df)

In [11]:
def prepare_fairness_df_for_model(model):
    male_confusion_matrix = confusion_matrix(prepare_y(income_df[income_df['gender'] == "Male"]), model.predict(X[X['gender_Male']]))
    female_confusion_matrix = confusion_matrix(prepare_y(income_df[income_df['gender'] == "Female"]), logistic.predict(X[~X['gender_Male']]))
    female_results = [f(male_confusion_matrix) for f in functions]
    male_results = [f(female_confusion_matrix) for f in functions]

    df = pd.DataFrame([female_results, male_results], columns=["Demographic Parity", "Equal Opportunity", "Positive Predictive Parity"], index=["Female", "Male"])
    df = df.T
    df["Coefficient"] = df["Female"] / df["Male"]
    return df

In [17]:
# Train logistic regression
logistic = LogisticRegression()
logistic.fit(X, y)
logistic_fairness_df = prepare_fairness_df_for_model(logistic)
logistic_fairness_df

,Female,Male,Coefficient
Demographic Parity,0.696233,0.890749,0.781627
Equal Opportunity,0.748124,0.917114,0.815737
Positive Predictive Parity,0.965071,0.970464,0.994443


In [21]:
# Train XGBoost
xgb_model = xgb.XGBClassifier()
xgb_model.fit(X, y)

xgb_fairness_df = prepare_fairness_df_for_model(xgb_model)
xgb_fairness_df

/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categoric

,Female,Male,Coefficient
Demographic Parity,0.696233,0.890749,0.781627
Equal Opportunity,0.883213,0.917114,0.963035
Positive Predictive Parity,0.934850,0.970464,0.963302


In [18]:
from sklearn.utils import resample

male_income_df = income_df[income_df['gender'] == "Male"]
female_income_df = income_df[income_df['gender'] == "Female"]

df_female_upsampled = resample(female_income_df, 
                               replace=True,
                               n_samples=len(male_income_df))
income_df_balanced = pd.concat([male_income_df, df_female_upsampled])
X_balanced = prepare_X(income_df_balanced)
y_balanced = prepare_y(income_df_balanced)

logistic_balanced = LogisticRegression()
logistic_balanced.fit(X_balanced, y_balanced)
logistic_balanced_fairness_df = prepare_fairness_df_for_model(logistic_balanced)
logistic_balanced_fairness_df

,Female,Male,Coefficient
Demographic Parity,0.696233,0.890749,0.781627
Equal Opportunity,0.743378,0.917114,0.810562
Positive Predictive Parity,0.974089,0.970464,1.003736


In [23]:
len(income_df), len(income_df_balanced)

(48842, 65300)

In [15]:
metrics = [f1_score, accuracy_score]
models = [
    ('Logistic Regression', logistic),
    ('XGBoost', xgb_model),
    ('Logistic Regression on balanced dataset', logistic_balanced)
]

# calculate results_df where models are rows and metrics are columns
results = []
for model_name, model in models:
    for metric in metrics:
        results.append({
            'Model': model_name,
            'Metric': metric.__name__,
            'Value': metric(y, model.predict(X))
        })

# make df from results
results_df = pd.DataFrame(results)
results_df = results_df.pivot(index="Model", columns="Metric", values="Value")
results_df

/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
/home/jan-praca/miniconda3/envs/XAI/lib/python3.11/site-packages/xgboost/data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, Categoric

Metric,accuracy_score,f1_score
Model,,
Logistic Regression,0.797899,0.380857
Logistic Regression on balanced dataset,0.797510,0.356120
XGBoost,0.896278,0.765723
